In [1]:
from binary_radix_tree import *
from morton_code import *
from check import *
from morton_code import *
from util import *
import random

In [23]:
num_points = 1000
num_bits_per_dim = 3
# Set random seed for reproducibility
random.seed(42)

# Generate random 3D points between 0 and 1
points = [(random.random(), random.random(), random.random()) for _ in range(num_points)]
# Filter to get only unique points
points = list(set(points))
print(f"num of unique points: {len(points)}")

# Convert points to Morton codes
morton_codes = []
for x, y, z in points:
    morton_code = coordinate_to_morton_code_3d(x, y, z, num_bits_per_dim)
    morton_codes.append(morton_code)

# Sort Morton codes
# Sort morton_codes and points together using morton_codes as the key
morton_codes, points = zip(*sorted(zip(morton_codes, points)))
morton_codes = list(morton_codes)
points = list(points)
# print(f"morton codes: {morton_codes}")
offset = range(len(morton_codes))

# get unique points
same = [1] + [0 if morton_codes[i] == morton_codes[i-1] else 1 for i in range(1, len(morton_codes))]
offset = [offset[i] for i in range(len(morton_codes)) if same[i] == 1] + [len(morton_codes)]
# print(f"offset: {offset}")
unique_mc = [morton_codes[i] for i in range(len(morton_codes)) if same[i] == 1]
# print(f"unique_mc: {unique_mc}")

# create binary radix tree 
brt = BinaryRadixTree(unique_mc, num_bits_per_dim * 3)

# test 
query_point = [random.random(), random.random(), random.random()]
# radius = random.random()
radius = 0.1
print(f"ref_point: {query_point}, radius: {radius}")
check = check_distance(query_point, radius, num_bits_per_dim)
query_mc = brt.traverse_tree_distance(check)

# print queried mc
query_points = []
for mc, shifts in query_mc.items():
    for cell_shift in shifts:
        # print(f"mc idx: {mc}, cell_shift: {cell_shift}")
        for idx in range(offset[mc], offset[mc + 1]):
            shifted_point = (points[idx][0] + cell_shift[0], 
                           points[idx][1] + cell_shift[1],
                           points[idx][2] + cell_shift[2])
            if check.check_point(shifted_point):
                query_points.append(shifted_point)

# check result 
bench_points = []
for shift_z in range(check.cell_shift_z_min, check.cell_shift_z_max + 1):
    for shift_y in range(check.cell_shift_y_min, check.cell_shift_y_max + 1):
        for shift_x in range(check.cell_shift_x_min, check.cell_shift_x_max + 1):
            for point in points:
                shifted_point = (point[0] + shift_x, point[1] + shift_y, point[2] + shift_z)
                if check.check_point(shifted_point):                    
                    bench_points.append(shifted_point)

# Sort both lists of points and compare them
sorted_query_points = sorted(query_points)
sorted_bench_points = sorted(bench_points)

print("Query and benchmark points match:", sorted_query_points == sorted_bench_points)

if sorted_query_points != sorted_bench_points:
    print("Number of query points:", len(sorted_query_points))
    print("Number of benchmark points:", len(sorted_bench_points))
    # Find points that are in one list but not the other
    query_set = set(sorted_query_points)
    bench_set = set(sorted_bench_points)
    print("Points in query but not in benchmark:", query_set - bench_set)
    print("Points in benchmark but not in query:", bench_set - query_set)



num of unique points: 1000
ref_point: [0.8640507204685769, 0.6509695631134773, 0.6871604042171144], radius: 0.1
Query and benchmark points match: False
Number of query points: 0
Number of benchmark points: 5
Points in query but not in benchmark: set()
Points in benchmark but not in query: {(0.8008168219994952, 0.7007256063215421, 0.735352868782368), (0.8184268595406696, 0.6135732805354648, 0.6426991638298314), (0.833744954639807, 0.703539925087371, 0.6116777657259501), (0.8289589728944645, 0.7387746721294828, 0.6854144402775763), (0.891425925810437, 0.6126523227617628, 0.7192739612759671)}
